In [11]:
import geopandas as gpd 
from geopandas import GeoDataFrame
import pandas as pd
import osmnx as ox 
import os 
import typing

In [12]:
CATEGORIES = {#"building": {"building": True}, 
              #"territories": {"boundary": "administrative", "admin_level": "2"}, 
              "landuse": {"landuse": True},
              "highway": {"highway": True},
              #"fire_station": {"amenity": ["fire_station"]}, 
              #"waterway": {"waterway": True}, 
              #"substation": {"power": ["substation"]}
}

In [13]:
 {‘amenity’:True, ‘landuse’:[‘retail’,’commercial’], ‘highway’:’bus_stop’}

SyntaxError: invalid character in identifier (Temp/ipykernel_15468/1843254095.py, line 1)

In [14]:
ox.config(use_cache=True, log_console=False, timeout=720)

In [28]:
def _append_poi_category(gdf_poi: GeoDataFrame,
                         tags: typing.Dict[str, list], 
                         col_poi_id: str = "id_poi"
                         ) -> GeoDataFrame: 
    """Append poi category from tag search to POI locations
    Args:
        gdf_poi (GeoDataFrame): POI locations
        tags (typing.Dict[str, list]): mapping containing osm key as key and list of osm values as value
        col_poi_id (str, optional): name of column uniquely identifying each POI. Defaults to "id_poi".
    Returns:
        GeoDataFrame: POI locations with POI category appended
    """    
    for key in tags: 
        if key not in set(gdf_poi.columns): 
            gdf_poi[key] = [None] * gdf_poi.shape[0]
    
    poi_category = pd.concat([gdf_poi.set_index(col_poi_id)[key] for key in tags]).dropna()
    allowed_values = [v for val in tags.values() for v in val]
    poi_category = poi_category[poi_category.isin(allowed_values)]
    poi_category = poi_category.groupby(col_poi_id).first()
    poi_category.name = "poi_cat"
    
    gdf_poi = gdf_poi.merge(poi_category, how="left", left_on=col_poi_id, right_index=True, validate="1:1")
    return gdf_poi


def _postprocess_osm_data(gdf_osm: gpd.GeoDataFrame, 
                          city_osm: str,
                          category: str,
                          cols_relevant: typing.List[str],
                          )  -> gpd.GeoDataFrame: 
    """Postprocess OSM POI locations: appending poi id and poi category, appending lat/lon coordinates, transform POI to 
    EPSG:25832
    
    Args:
        gdf_osm (GeoDataFrame): POI locations
        tags (typing.Dict[str, list]): mapping containing osm key as key and list of osm values as value
        cols_relevant (typing.List[str], optional): list of relevant columns. If not specified all columns are returned. Defaults to ["geometry", "id_poi", "category", "longitude", "latitude", "city"].

    Returns:
        GeoDataFrame: [description]
    """    
    gdf_osm["city"] = city_osm
    gdf_osm.reset_index(drop=False, inplace=True)
    
    gdf_osm["osm_id"] = gdf_osm["element_type"] + "/" + gdf_osm["osmid"].astype(str)
    gdf_osm.drop(columns=["ways", "nodes"], errors="ignore", inplace=True)
        
    if cols_relevant is None: 
        cols_relevant = gdf_osm.columns 
    
    repr_point =  gdf_osm.representative_point()
    gdf_osm["longitude"] = repr_point.x
    gdf_osm["latitude"] = repr_point.y
    gdf_osm["category"] = category 

    gdf_osm.to_crs("EPSG:25832", inplace=True)
    
    return gdf_osm[cols_relevant]

def _get_osm_category(category: str, tags: dict, city: str, cols_relevant=["geometry", "osm_id", "category", "longitude", "latitude", "city"])  -> gpd.GeoDataFrame: 
    gdf_osm = ox.geometries_from_place(f"{city}, Germany", tags=tags)
    gdf_osm = _postprocess_osm_data(gdf_osm, city_osm=city, category=category, cols_relevant=cols_relevant)
    
    return gdf_osm


def retrieve_osm_data(categories: dict, dir_save: str = "./data", city: str = "Essen", cols_relevant=["geometry", "osm_id", "category", "longitude", "latitude", "city"]): 
    for cat, tags in categories.items(): 
        print(cat)
        gdf_osm = _get_osm_category(category=cat, tags=tags, city=city, cols_relevant=cols_relevant)
        fullpath_save = os.path.join(dir_save, f"{cat}.geojson")
        
        gdf_osm.to_file(fullpath_save, driver="GeoJSON")

In [34]:
CATEGORIES = {#"building": {"building": True}, 
              #"territories": {"boundary": "administrative", "admin_level": "2"}, 
              #"landuse": {"landuse": True},
              "highway": {"highway": True},
              #"fire_station": {"amenity": ["fire_station"]}, 
              #"waterway": {"waterway": True}, 
              #"substation": {"power": ["substation"]}
}

In [35]:
%%time
retrieve_osm_data(categories=CATEGORIES, cols_relevant=["geometry", "osm_id", "category", "longitude", "latitude", "city", "name", "highway"])

highway


C:\Users\nblo\anaconda3\envs\mega\lib\site-packages\geopandas\array.py:524: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.representative_point(self.data), crs=self.crs)
C:\Users\nblo\anaconda3\envs\mega\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\nblo\anaconda3\envs\mega\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):

Wall time: 1min 41s


In [36]:
import glob


osm_files = glob.glob("./data/waterway.geojson") 
gdf_osm = gpd.read_file(osm_files[0])



In [37]:
gdf_osm

,osm_id,category,longitude,latitude,city,name,waterway,geometry
0,node/280950170,waterway,7.001391,51.397096,Essen,None,lock_gate,POINT (360965.126 5695880.324)
1,node/280950172,waterway,7.001216,51.396880,Essen,None,lock_gate,POINT (360952.279 5695856.705)
2,node/286048608,waterway,7.003679,51.488172,Essen,None,weir,POINT (361400.262 5706003.420)
3,node/290111349,waterway,7.060359,51.394777,Essen,Paulsen Werft,boatyard,POINT (365060.053 5695512.272)
4,node/472613567,waterway,7.113099,51.428779,Essen,None,lock_gate,POINT (368826.239 5699197.484)
...,...,...,...,...,...,...,...,...
949,way/977516096,waterway,7.044821,51.414333,Essen,Schellenberger Bach,stream,"LINESTRING (364035.532 5697714.307, 364037.195..."
950,way/977516097,waterway,7.040991,51.412951,Essen,Schellenberger Bach,stream,"LINESTRING (363505.322 5697374.410, 363511.238..."
951,way/1013337144,waterway,7.038469,51.485791,Essen,None,ditch,"LINESTRING (363800.823 5705695.482, 363804.305..."
952,way/1017348298,waterway,6.940428,51.376316,Essen,Brederbach,stream,"LINESTRING (356666.905 5693690.630, 356659.606..."


In [15]:
import configparser 
import sqlalchemy

CONFIG = "config.cfg"
ENGINE = "engine"
URL = "url"
PORT = "port"
USERNAME = "db_username"
PASSWORD = "password"
DBNAME = "db_name"
DRIVER = "driver" 
FSTR_POSTGIS_CONNECT = f"{{{ENGINE}}}://{{{USERNAME}}}:{{{PASSWORD}}}@{{{URL}}}:{{{PORT}}}/{{{DBNAME}}}"
config = configparser.ConfigParser()
config.read(CONFIG)

engine = sqlalchemy.create_engine(FSTR_POSTGIS_CONNECT.format(**config["postgis"]), echo=False)

In [30]:
osm_files = glob.glob("./data/territories.geojson")

In [31]:
schema = "deep_dive" 

engine.execute(f"CREATE SCHEMA IF NOT EXISTS {schema};")
for of in osm_files:
    gdf_osm = gpd.read_file(of)
    table_name = gdf_osm["category"].iloc[0]
    
    gdf_osm.to_postgis(con=engine, name=table_name, schema=schema, if_exists="replace")

In [32]:
gdf_osm

,osm_id,category,longitude,latitude,city,geometry
0,way/28652989,territories,7.030982,51.522565,Essen,"LINESTRING (362789.288 5709408.686, 362806.052..."
1,way/28653251,territories,7.072503,51.498703,Essen,"LINESTRING (365828.177 5706931.739, 365999.977..."
2,way/28663860,territories,7.057337,51.515593,Essen,"LINESTRING (364430.698 5708546.687, 364452.892..."
3,way/28664785,territories,7.098284,51.487907,Essen,"LINESTRING (368061.202 5706683.340, 368034.262..."
4,way/29411517,territories,7.108027,51.376771,Essen,"LINESTRING (368975.938 5693887.906, 368967.479..."
...,...,...,...,...,...,...
347,relation/9050091,territories,6.970857,51.512263,Essen,"POLYGON ((357358.566 5708418.429, 357663.469 5..."
348,relation/9050092,territories,6.944933,51.502866,Essen,"POLYGON ((356187.210 5707215.869, 356249.312 5..."
349,relation/9263708,territories,6.857030,51.472060,Essen,"POLYGON ((347854.796 5702760.643, 347801.356 5..."
350,relation/9263978,territories,6.882660,51.514661,Essen,"POLYGON ((350756.653 5706646.965, 350729.394 5..."
